In [31]:
import pandas as pd
import numpy as np
import datetime

import seaborn as sn

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [32]:
claims_df = pd.read_csv('Claims1/claims.csv', sep= '|', nrows = 100000)

members_df = pd.read_csv('Claims1/members.csv', sep= '|', nrows = 100000)

In [33]:
claims_df = claims_df.merge(members_df, how = 'left', left_on = 'members_id', right_on = 'id')

In [36]:
print(claims_df.shape)
claims_df = claims_df[claims_df['event_date'] <= '2019-01-01T00:00:00Z']
print(claims_df.shape)

(100000, 98)
(94461, 98)


In [44]:
claims_df['event_date'] = claims_df.event_date.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ"))

In [46]:
claims_df['event_month'] = claims_df['event_date'].dt.month


In [56]:
claims_df['event_quater'] = pd.cut(claims_df.event_month, [0,3,6,9,12], labels = [1,2,3,4])


In [63]:
claims_df['top_5_pot_cd_23'] = (claims_df.pot_cd == 23)*1
claims_df['top_5_pot_cd_41'] = (claims_df.pot_cd == 41)*1
claims_df['top_5_pot_cd_81'] = (claims_df.pot_cd == 81)*1
claims_df['top_5_net_paid_amt_for_rx_claims'] = claims_df.net_paid_amt[claims_df.event_type == 'rxclaim']

In [177]:
temp_df = claims_df.groupby(['members_id', 'event_quater'])['top_5_pot_cd_23', 
                                                  'top_5_pot_cd_41', 
                                                  'top_5_pot_cd_81', 
                                                  'top_5_net_paid_amt_for_rx_claims'].sum().reset_index()



# gby.top_5_pot_cd_23.sum().reset_index(drop = True).describe()
#     'top_5_pot_cd_23': ['sum', 'max']}).reset_index()

In [178]:
temp_df.head()

,members_id,event_quater,top_5_pot_cd_23,top_5_pot_cd_41,top_5_pot_cd_81,top_5_net_paid_amt_for_rx_claims
0,001e2eaec4cf9a76,1,0.0,0.0,0.0,0.0
1,001e2eaec4cf9a76,2,0.0,0.0,0.0,0.0
2,001e2eaec4cf9a76,3,0.0,0.0,7.0,70.0
3,001e2eaec4cf9a76,4,0.0,0.0,0.0,163.0
4,003b7dc36c24d282,1,NaN,NaN,NaN,NaN


In [179]:
temp_df

,members_id,event_quater,top_5_pot_cd_23,top_5_pot_cd_41,top_5_pot_cd_81,top_5_net_paid_amt_for_rx_claims
0,001e2eaec4cf9a76,1,0.0,0.0,0.0,0.0
1,001e2eaec4cf9a76,2,0.0,0.0,0.0,0.0
2,001e2eaec4cf9a76,3,0.0,0.0,7.0,70.0
3,001e2eaec4cf9a76,4,0.0,0.0,0.0,163.0
4,003b7dc36c24d282,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5803,ffaf7b0680b0cf58,4,NaN,NaN,NaN,NaN
5804,ffd7425f501b48a3,1,NaN,NaN,NaN,NaN
5805,ffd7425f501b48a3,2,NaN,NaN,NaN,NaN
5806,ffd7425f501b48a3,3,NaN,NaN,NaN,NaN


In [180]:
lead_temp_df = temp_df.groupby(['members_id'])['top_5_pot_cd_23', 
                                                  'top_5_pot_cd_41', 
                                                  'top_5_pot_cd_81', 
                                                  'top_5_net_paid_amt_for_rx_claims'].shift(1).reset_index()


temp_df.rename(columns={"top_5_pot_cd_23": "predictors_quaterly_sums_top_5_pot_cd_23", 
                 "top_5_pot_cd_41": "predictors_quaterly_sums_top_5_pot_cd_41", 
                 "top_5_pot_cd_81": "predictors_quaterly_sums_top_5_pot_cd_81",
                 "top_5_net_paid_amt_for_rx_claims": "predictors_quaterly_sums_top_5_net_paid_amt_for_rx_claims"}, inplace = True)


In [181]:
lead_temp_df.head()

,index,top_5_pot_cd_23,top_5_pot_cd_41,top_5_pot_cd_81,top_5_net_paid_amt_for_rx_claims
0,0,NaN,NaN,NaN,NaN
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,0.0,0.0,7.0,70.0
4,4,NaN,NaN,NaN,NaN


In [191]:
final_data = pd.concat([temp_df, lead_temp_df], axis = 1)
final_data.drop('index', inplace = True, axis = 1)
final_data = final_data[final_data.event_quater > 1]

In [192]:
final_data.head(20)

,members_id,event_quater,predictors_quaterly_sums_top_5_pot_cd_23,predictors_quaterly_sums_top_5_pot_cd_41,predictors_quaterly_sums_top_5_pot_cd_81,predictors_quaterly_sums_top_5_net_paid_amt_for_rx_claims,top_5_pot_cd_23,top_5_pot_cd_41,top_5_pot_cd_81,top_5_net_paid_amt_for_rx_claims
1,001e2eaec4cf9a76,2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,001e2eaec4cf9a76,3,0.0,0.0,7.0,70.00,0.0,0.0,0.0,0.00
3,001e2eaec4cf9a76,4,0.0,0.0,0.0,163.00,0.0,0.0,7.0,70.00
5,003b7dc36c24d282,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,003b7dc36c24d282,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,003b7dc36c24d282,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,009b5451dd6c8634,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,009b5451dd6c8634,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,009b5451dd6c8634,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,009ee2c9ad3369c5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 1. ER Visits--> pot_cd--> 23
# 2.Land Ambulance --> pot_cd-->41
# 3. LAB --> pot_cd-->81
# 4.Rx Claims--> event_type==‘rxclaim’--> Net Amt/Total RX Claims